In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Reading File
file_path = r'20230906_HMspeed.csv'
hundredmile_speed = pd.read_csv(file_path)
file_path = r'MaxOD_Yong.csv'
ETC_data_1 = pd.read_csv(file_path)
file_path = r'MaxOD_Hang.csv'
ETC_data_2 = pd.read_csv(file_path)
file_path = r'gantry.csv'
gantry_data = pd.read_csv(file_path)
file_path = r'station.csv'
station_data = pd.read_csv(file_path)
file_path = r'flow_num10.csv'
Volume_data = pd.read_csv(file_path)

Data Preprocessing

In [ ]:
'''Hundred Mile Data'''
hundredmile_speed['time_flag'] = pd.to_datetime(hundredmile_speed['time_flag'])
df_yong_raw = hundredmile_speed[hundredmile_speed['direction'] == 1].copy()
df_hu_raw = hundredmile_speed[hundredmile_speed['direction'] == 2].copy()

''' Volume Data '''
Volume_data['start_time'] =  pd.to_datetime(Volume_data['start_time'])
Volume_data= Volume_data[['start_time', 'gantryid', 'count']]
Volume_data = Volume_data[Volume_data['start_time'].dt.date == pd.to_datetime('2023-09-06').date()]
Volume_data = Volume_data[Volume_data['gantryid'].str.len() > 15]
Volume_data['hour'] = Volume_data['start_time'].dt.floor('60min')

'''ETC Data'''
#车型对应
def def_class(vehicletype):
    if int(vehicletype) in [1, 2]:
        return '1'
    elif int(vehicletype) in [3, 4]:
        return '3'
    else:
        return '1'
ETC_data_1['vehicleType'] = ETC_data_1['vehicleType'].apply(def_class)
ETC_data_2['vehicleType'] = ETC_data_2['vehicleType'].apply(def_class)

def calculate_hourly_speed_average(df):
    df['startTime'] = pd.to_datetime(df['startTime'])
    df['speed'] = (df['distance']*3600) / df['time_interval']
    df['hour'] = df['startTime'].dt.floor('60min')
    hourly_average = df.groupby('hour')['speed'].mean()

    return hourly_average

average_result1 = calculate_hourly_speed_average(ETC_data_1)
average_result2 = calculate_hourly_speed_average(ETC_data_2)

ETC_data_1['hour'] = ETC_data_1['startTime'].dt.floor('60min')
ETC_data_2['hour'] = ETC_data_2['startTime'].dt.floor('60min')

def select_top_speed_hours(df, threshold_speed=90):
    select_hours = df[df > threshold_speed].index
    return select_hours

select_times1 = select_top_speed_hours(average_result1)
select_times1 = pd.DataFrame(select_times1)
select_times2 = select_top_speed_hours(average_result2)
select_times2 = pd.DataFrame(select_times2)

freerun_data1 = pd.merge(ETC_data_1, select_times1, on='hour', how='inner')
freerun_data2 = pd.merge(ETC_data_2, select_times2, on='hour', how='inner')

Data Selection

In [ ]:
Volume_count = Volume_data.groupby(['gantryid', Volume_data['start_time'].dt.floor('H')])['count'].sum().reset_index()
lane_num = 4 
capacity = 2200
Volume_count['vc_rate'] = Volume_count['count']/(lane_num*capacity)

def calculate_los(vc_rate):
    if 0 < vc_rate <= 0.34:
        return 1
    elif 0.34 < vc_rate <= 0.74:
        return 2
    elif 0.74 < vc_rate < 0.88:
        return 3
    else:
        return 4


Volume_count['LOS'] = Volume_count['vc_rate'].apply(calculate_los)
Volume_count = pd.merge(Volume_count, gantry_data, left_on='gantryid', right_on='gantryId',how='inner')
Volume_count= Volume_count[['start_time', 'gantryid', 'count','LOS','direction']]
Volume_count1 = Volume_count[Volume_count['direction'] == 1]
Volume_count2 = Volume_count[Volume_count['direction'] == 2]

def time_data_selection(df):
    hourly_count = df.groupby(df['start_time'].dt.hour)['count'].sum()
    df['start_hour'] = df['start_time'].dt.hour
    s_time = df.groupby('start_hour').filter(lambda x: (x['LOS'] != 1).all())['start_hour'].unique()
    if len(s_time) < 3:
        top_hours = hourly_count.sort_values(ascending=False).head(3).index
        for hour in top_hours:
            if hour not in s_time:
                s_time = np.append(s_time, hour)
                if len(s_time) == 3:
                    break
    
    select_times = s_time

    return select_times

s1_times = time_data_selection(Volume_count1)
s2_times = time_data_selection(Volume_count2)
s1_times = pd.DataFrame({'hour': s1_times})
s2_times = pd.DataFrame({'hour': s2_times})

# Selection
freerun_data1['hour'] = freerun_data1['startTime'].dt.hour
freerun_data2['hour'] = freerun_data2['startTime'].dt.hour
freerun_LOS1 = pd.merge(freerun_data1, s1_times, on='hour', how='inner')
freerun_LOS2 = pd.merge(freerun_data2, s2_times, on='hour', how='inner')

def Congestion_Detection(df):

    # Calculate Mean Speed
    df_hour_average = df.groupby(['start_mileage', pd.Grouper(key='time_flag', freq='60min')]).agg({'speed': 'mean', 'end_mileage': 'first'}).reset_index()
    threshold = 60
    df_hour_average['is_congested'] = df_hour_average['speed'].apply(lambda x: 1 if x < threshold else 0)
    
    # get congestion hour
    congested_hours = pd.DataFrame({'hour': df_hour_average.loc[df_hour_average['is_congested'] == 1, 'time_flag'].dt.hour.unique()})
    all_hours = set(range(24))
    congested_hour_set = set(congested_hours['hour'])
    non_congested_hours = pd.DataFrame({'hour': list(all_hours - congested_hour_set)})

    return non_congested_hours

Fluent_time_yong = Congestion_Detection(df_yong_raw)
Fluent_time_hu = Congestion_Detection(df_hu_raw)

freerun_yong = pd.merge(freerun_LOS1, Fluent_time_yong, on='hour', how='inner')
freerun_hang = pd.merge(freerun_LOS2, Fluent_time_hu, on='hour', how='inner')

GMM Clustering

In [ ]:


def perform_gmm_clustering(data, features, n_components):
    X = data[features].values
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    data['cluster_labels'] = gmm.fit_predict(X)
    return data

def plot_gmm_clusters_with_stats(data, title):
    plt.figure(figsize=(8, 6))
    for cluster in data['cluster_labels'].unique():
        cluster_data = data[data['cluster_labels'] == cluster]
        plt.scatter(cluster_data['speed'], cluster_data['cluster_labels'], label=f'Cluster {cluster}')
        std_deviation = np.std(cluster_data['speed'])
        max_speed = np.max(cluster_data['speed'])
        min_speed = np.min(cluster_data['speed'])
        plt.text((max_speed + min_speed) / 2, cluster, f'Max={max_speed:.2f}\nMin={min_speed:.2f}\nStdDev={std_deviation:.2f}', fontsize=10, verticalalignment='bottom', color='red')

    plt.title(title)
    plt.xlabel('Speed')
    plt.ylabel('Cluster')
    plt.legend()
    plt.show()


# Clustering for Small vehs in Ningbo Direction
vehicle_data_type_1 = freerun_yong[freerun_yong['vehicleType'] == '1']
vehicle_data_type_1 = perform_gmm_clustering(vehicle_data_type_1, features=['speed'], n_components=3)
plot_gmm_clusters_with_stats(vehicle_data_type_1, f'GMM Clustering Results (freerun_yong, vehicle_type=1)')

# Clustering for Large vehs in Ningbo Direction
vehicle_data_type_3 = freerun_yong[freerun_yong['vehicleType'] == '3']
vehicle_data_type_3 = perform_gmm_clustering(vehicle_data_type_3, features=['speed'], n_components=3)
plot_gmm_clusters_with_stats(vehicle_data_type_3, f'GMM Clustering Results (freerun_yong, vehicle_type=3)')
